# SP Simulation of topology with slack, line and VSI

In [ ]:
from villas.dataprocessing.readtools import *
from villas.dataprocessing.timeseries import *
import matplotlib.pyplot as plt
import re
import dpsimpy

# %matplotlib widget

## SP Simulation

### Parameterization

In [ ]:
final_time = 4.0
time_step = 1e-3
sim_name = "SP_Slack_PiLine_VSI_with_PF_Init"
pv_with_control = True
cmd_scale_P = 1.0
cmd_scale_I = 1.0
V_nom = 20e3

### Powerflow for Initialization

In [ ]:
time_step_pf = final_time
final_time_pf = final_time
sim_name_pf = sim_name + "_PF"
dpsimpy.Logger.set_log_dir('logs/' + sim_name_pf)

# Components
n1_pf = dpsimpy.sp.SimNode('n1', dpsimpy.PhaseType.Single)
n2_pf = dpsimpy.sp.SimNode('n2', dpsimpy.PhaseType.Single)

extnet_pf = dpsimpy.sp.ph1.NetworkInjection('Slack', dpsimpy.LogLevel.debug)
extnet_pf.set_parameters(voltage_set_point=V_nom)
extnet_pf.set_base_voltage(V_nom)
extnet_pf.modify_power_flow_bus_type(dpsimpy.PowerflowBusType.VD)

line_pf = dpsimpy.sp.ph1.PiLine('PiLine', dpsimpy.LogLevel.debug)
line_pf.set_parameters(R=0.5*5, L=0.5/314*5, C=50e-6/314*5)
line_pf.set_base_voltage(V_nom)

load_pf = dpsimpy.sp.ph1.Load('Load', dpsimpy.LogLevel.debug)
load_pf.set_parameters(active_power=-100e3, reactive_power=-50e3, nominal_voltage=20e3)
load_pf.modify_power_flow_bus_type(dpsimpy.PowerflowBusType.PQ)

# Topology
extnet_pf.connect([n1_pf])
line_pf.connect([n1_pf, n2_pf])
load_pf.connect([n2_pf])
system_pf = dpsimpy.SystemTopology(50, [n1_pf, n2_pf], [extnet_pf, line_pf, load_pf])

# Logging
logger_pf = dpsimpy.Logger(sim_name_pf)
logger_pf.log_attribute('v1', 'v', n1_pf)
logger_pf.log_attribute('v2', 'v', n2_pf)

# Simulation
sim_pf = dpsimpy.Simulation(sim_name_pf, dpsimpy.LogLevel.debug)
sim_pf.set_system(system_pf)
sim_pf.set_time_step(time_step_pf)
sim_pf.set_final_time(final_time_pf)
sim_pf.set_domain(dpsimpy.Domain.SP)
sim_pf.set_solver(dpsimpy.Solver.NRP)
sim_pf.do_init_from_nodes_and_terminals(False)
sim_pf.add_logger(logger_pf)
sim_pf.run()

### Dynamic Simulation

In [ ]:
time_step_sp = time_step
final_time_sp = final_time + time_step_sp
sim_name_sp = sim_name + "_SP"
dpsimpy.Logger.set_log_dir('logs/' + sim_name_sp)

# Components
gnd = dpsimpy.sp.SimNode.gnd
n1_sp = dpsimpy.sp.SimNode('n1', dpsimpy.PhaseType.Single)
n2_sp = dpsimpy.sp.SimNode('n2', dpsimpy.PhaseType.Single)

extnet_sp = dpsimpy.sp.ph1.NetworkInjection('Slack', dpsimpy.LogLevel.debug)
extnet_sp.set_parameters(complex(V_nom, 0))

line_sp = dpsimpy.sp.ph1.PiLine('PiLine', dpsimpy.LogLevel.debug)
line_sp.set_parameters(R=0.5*5, L=(0.5/314)*5, C=(50e-6/314)*5)

pv = dpsimpy.sp.ph1.AvVoltageSourceInverterDQ("pv", "pv", dpsimpy.LogLevel.debug, with_trafo=True)
pv.set_parameters(sys_omega=2 * np.pi * 50, sys_volt_nom=V_nom, p_ref=100e3, q_ref=50e3)
pv.set_controller_parameters(Kp_pll=cmd_scale_P * 0.25, Ki_pll=cmd_scale_I * 0.2,
                             Kp_power_ctrl=cmd_scale_P *0.001, Ki_power_ctrl=cmd_scale_I * 0.008 ,
                             Kp_curr_ctrl=cmd_scale_P * 0.3, Ki_curr_ctrl=cmd_scale_I * 1, omega_cutoff=2*np.pi*50)
pv.set_filter_parameters(Lf=0.002,Cf=789.3e-6,Rf=0.1,Rc=0.1)
pv.set_transformer_parameters(nom_voltage_end_1=V_nom, nom_voltage_end_2=1500, rated_power = 5e6,
                              ratio_abs=V_nom / 1500, ratio_phase=0,resistance=0, inductance=0.928e-3)
pv.set_initial_state_values(p_init=100e3, q_init=50e3, phi_d_init=0, phi_q_init=0, gamma_d_init=0, gamma_q_init=0)
pv.with_control(pv_with_control)

# Topology
extnet_sp.connect([n1_sp])
line_sp.connect([n1_sp, n2_sp])
pv.connect([n2_sp])
system_sp = dpsimpy.SystemTopology(50, [n1_sp, n2_sp], [extnet_sp, line_sp, pv])

# Initialization of dynamic topology
system_sp.init_with_powerflow(system_pf)

# Logging
logger_sp = dpsimpy.Logger(sim_name_sp)
logger_sp.log_attribute('v1', 'v', n1_sp)
logger_sp.log_attribute('v2', 'v', n2_sp)
logger_sp.log_attribute('i12', 'i_intf', line_sp)


input_names = [
    "pv_powerctrl_input_pref", "pv_powerctrl_input_qref", "pv_powerctrl_input_vcd",
    "pv_powerctrl_input_vcq", "pv_powerctrl_input_ircd", "pv_powerctrl_input_ircq"
]
logger_sp.log_attribute(input_names, 'powerctrl_inputs', pv)

state_names = [
    "pv_powerctrl_state_p", "pv_powerctrl_state_q", "pv_powerctrl_state_phid",
    "pv_powerctrl_state_phiq", "pv_powerctrl_state_gammad", "pv_powerctrl_state_gammaq"
]
logger_sp.log_attribute(state_names, 'powerctrl_states', pv)

output_names = [
    "pv_powerctrl_output_vsd", "pv_powerctrl_output_vsq"
]

logger_sp.log_attribute(output_names, 'powerctrl_outputs', pv)

logger_sp.log_attribute('pv_v_intf', 'v_intf', pv)
logger_sp.log_attribute('pv_i_intf', 'i_intf', pv)
logger_sp.log_attribute('pv_pll_output', 'pll_output', pv)
logger_sp.log_attribute('pv_vsref', 'Vsref', pv)
logger_sp.log_attribute('pv_vs', 'Vs', pv)

# load step sized in absolute terms
load_switch = dpsimpy.sp.ph1.Switch("Load_Add_Switch_n2", dpsimpy.LogLevel.debug)
connection_node = system_sp.node('n2')
resistance = np.abs(connection_node.initial_single_voltage())**2 / 10e6
load_switch.set_parameters(1e9, resistance)
load_switch.open()
system_sp.add(load_switch)
system_sp.connect_component(load_switch, [gnd, system_sp.node('n2')])
logger_sp.log_attribute('switchedload_i', 'i_intf', load_switch)
load_step_event = dpsimpy.event.SwitchEvent(np.round(5.0/time_step)*time_step, load_switch, True)

# Simulation
sim_sp = dpsimpy.Simulation(sim_name_sp, dpsimpy.LogLevel.debug)
sim_sp.set_system(system_sp)
sim_sp.set_time_step(time_step_sp)
sim_sp.set_final_time(final_time_sp)
sim_sp.set_domain(dpsimpy.Domain.SP)
sim_sp.add_event(load_step_event)
sim_sp.add_logger(logger_sp)
sim_sp.run()

## PF results

In [ ]:
modelName = 'SP_Slack_PiLine_VSI_with_PF_Init_PF'
path = 'logs/' + modelName + '/'
dpsim_result_file = path + modelName + '.csv'

ts_dpsim_pf = read_timeseries_csv(dpsim_result_file)

## SP results

In [ ]:
modelName = 'SP_Slack_PiLine_VSI_with_PF_Init_SP'
path = 'logs/' + modelName + '/'
dpsim_result_file = path + modelName + '.csv'

ts_dpsim = read_timeseries_csv(dpsim_result_file)

In [ ]:
plt.figure(figsize=(12,6))
for ts_name, ts_obj  in ts_dpsim.items():
    if ts_name == 'v1' or ts_name == 'v2':
        plt.plot(ts_obj.time, ts_obj.abs().values, label=ts_name)
for ts_name, ts_obj  in ts_dpsim_pf.items():
    if ts_name == 'v1' or ts_name == 'v2':
        plt.plot(ts_obj.time, ts_obj.abs().values, label=ts_name+'_pf', linestyle=':')
plt.legend()
plt.show()